### Notebook for checking Kasthuri 2015 - Claims 2 Neurons
GitRepo reference: https://github.com/neurodata/kasthuri2015. Claims list can be found in the "Issues Tab" 
Useful link for figuring out the channels available to you https://api.bossdb.io/v1/mgmt/resources/Kasthuri2015/em 

In [1]:
# Import Kasthuri2015/em data using Intern. Import the relevant annotated data (neuron, mitochondria, synapses)
from intern import array
import numpy as np
import backoff
import requests 
from tqdm import tqdm
from datetime import datetime
from joblib import Parallel, delayed

import matplotlib.pyplot as plt

%matplotlib widget

In [7]:
# Print the shape of the data and plot a slice with the annotation data superimposed to check that things are aligned. 
k_em = array("bossdb://kasthuri2015/em/cc")
k_neuron = array("bossdb://kasthuri2015/em/3cylneuron_v1")

emS = k_em.shape
neZ, neY, neX = k_neuron.shape

print("The EM data is of shape", emS)
print("The Neuron data is of shape (", neZ, neY, neX, ")")

/Users/riverjl1/opt/anaconda3/envs/intern/lib/python3.10/site-packages/intern/convenience/array.py:449: ImportWarning: CloudVolume is not installed. Accessing channel using CVDB.
  warnings.warn("CloudVolume is not installed. Accessing channel using CVDB.", ImportWarning)


The EM data is of shape (1849, 13312, 10752)
The Neuron data is of shape ( 1849 13312 10752 )


In [3]:
# neuron counter function that uses parallel to run multiple batches of counting at once. 

@backoff.on_exception(backoff.expo, 
                      requests.exceptions.RequestException, 
                      max_tries=3)
def neuron_count(neuronData:array, zBound:int, yBound:int, xBound:int, zBase=0, yBase=0, xBase=0) -> int:
    uni_val = np.ndarray([])
    # tqdm used for timing on smaller segments to test timing. 
    def neuron_count(i):
        np.append(uni_val, np.unique(neuronData[i, yBase:yBound, xBase:xBound]))
        return np.unique(uni_val)
    all_values = Parallel(n_jobs=20)(delayed(neuron_count)(i) for i in range(zBase,zBound))
    
    return np.unique(np.concatenate(all_values))

#claim_labels = neuron_count(k_neuron, neZ//2, neY//10, neX//10)

In [5]:
# Bin maker for cycling through stacks of slices 
def split(upper_lim, n_bins):
    k, m = divmod(upper_lim, n_bins)
    return [(i*k+min(i, m),(i+1)*k+min(i+1, m)) for i in range(n_bins)]


In [9]:
# Cell that executes counting in a for loop.  
all_values = []

for x_bound_lower, x_bound_upper in tqdm(split(neX,10)):
    for y_bound_lower, y_bound_upper in split(neY,10):
        for z_bound_lower, z_bound_upper in split(neZ,2):
            all_values.append(neuron_count(k_neuron, z_bound_upper, y_bound_upper, x_bound_upper, z_bound_lower, y_bound_lower, x_bound_lower))


 20%|██        | 2/10 [1:47:34<7:10:18, 3227.31s/it]


HTTPError: Get cutout failed on 3cylneuron_v1, got HTTP response: (504) - <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body>
</html>


In [ ]:
# Plotting Cell
em_slice = k_em[1500, 9500:10000, 9500:10000]
neuron_slice = k_neuron[1500, 9500:10000, 9500:10000]

fig, ax = plt.subplots(figsize=(5,5))
plt.imshow(em_slice, cmap="Greys_r")
plt.imshow(neuron_slice, alpha=0.5, cmap="tab20")
